In [2]:
pip install pyspark

     |████████████████████████████████| 204.2MB 75kB/s 
     |████████████████████████████████| 204kB 44.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=992facfff156b740351240c75bdf3840287e35ba5a29968b67002bb9031b3c0e
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
from pyspark import SparkContext
import time
sc = SparkContext()

In [33]:
filepath = '/content/drive/My Drive/Coursework/CS 5683_Big_Data/Assignment-6/email-Eu-core.txt'
links = sc.textFile(filepath)

links.repartition(32)

links_formatted = links.map(lambda x: (int(x.split(' ')[0]),int(x.split(' ')[1])))

print(links_formatted.count())
print(links_formatted.take(3))


25571
[(0, 1), (2, 3), (2, 4)]


In [34]:
auths = links_formatted.flatMap(lambda x: x).distinct().map(lambda x: (x, 1.0))
hubs = auths

print(auths.count())
print(auths.take(3))

print(hubs.count())
print(hubs.take(3))

1005
[(0, 1.0), (2, 1.0), (4, 1.0)]
1005
[(0, 1.0), (2, 1.0), (4, 1.0)]


In [35]:
# NORMALIZE
import numpy as np

def normalize(rdd):
  temp = rdd.map(lambda x: ('sum', x[1]**2)).reduceByKey(lambda x, y: x+y)

  norm = temp.collect()[0][1] ** 0.5
  updated = rdd.map(lambda x: (x[0], x[1] / norm))
  #updated = updated.sortBy(lambda x: x[1], ascending=False)

  return updated

links_formatted.cache()

for i in range(5):
  start = time.time()
  
  # UPDATE AUTHS
  auths = links_formatted.join(hubs).map(lambda x: (x[1][0], x[1][1])).reduceByKey(lambda x, y: x + y)

  # UPDATE HUBS
  hubs = links_formatted.map(lambda x: (x[1], x[0])).join(auths).map(lambda x: (x[1][0], x[1][1])).reduceByKey(lambda x, y: x + y)

  auths = normalize(auths)
  hubs = normalize(hubs)

  auths.cache()
  hubs.cache()
  
  print(i, np.round(time.time() - start,4))

  auths_sort = auths.sortBy(lambda x: x[1], ascending=False)
  hubs_sort = hubs.sortBy(lambda x: x[1], ascending=False)

  print(auths_sort.take(3))
  print(hubs_sort.take(3))



0 2.0549
[(160, 0.1769052195768834), (62, 0.1493680863408591), (107, 0.14102350051176082)]
[(160, 0.18994196271725522), (82, 0.16127271673704585), (121, 0.16024834606070307)]
1 3.1797
[(160, 0.14750003475410997), (107, 0.13808403387793916), (62, 0.1348562355943082)]
[(160, 0.19243444120886893), (82, 0.17204749498461142), (121, 0.17031985053906945)]
2 4.4417
[(160, 0.14427365230617642), (107, 0.1374083820471531), (62, 0.1334985204678465)]
[(160, 0.1917871155620407), (82, 0.1731736171493218), (121, 0.17155848718625175)]
3 5.3084
[(160, 0.1439311906422588), (107, 0.13743170593626355), (62, 0.13342657656874993)]
[(160, 0.19160218541639626), (82, 0.17329807781661918), (121, 0.17172697934032477)]
4 6.0724
[(160, 0.1438931225218173), (107, 0.1374561503810528), (62, 0.1334311702675352)]
[(160, 0.19156208773443828), (82, 0.17331073340920658), (121, 0.17175131608730435)]


In [11]:
auths = auths.sortBy(lambda x: x[1], ascending=False)
hubs = hubs.sortBy(lambda x: x[1], ascending=False)

print(auths.take(5))
print(hubs.take(5))

[(160, 0.1438931225218173), (107, 0.1374561503810528), (62, 0.1334311702675352), (434, 0.12923031627427153), (121, 0.12895469745158045)]
[(160, 0.19156208773443828), (82, 0.17331073340920658), (121, 0.17175131608730435), (107, 0.1583726855711284), (62, 0.1483661159857358)]


In [19]:
print(auths.take(5))
print(hubs.take(5))

[(160, 0.14427365230617642), (107, 0.1374083820471531), (62, 0.1334985204678465), (434, 0.1291914566822873), (121, 0.12878986769481088)]
[(160, 0.1917871155620407), (82, 0.1731736171493218), (121, 0.17155848718625175), (107, 0.15817067382752073), (62, 0.14825338393737628)]


In [15]:
print(auths.take(5))
print(hubs.take(5))

[(160, 0.1438931225218173), (107, 0.1374561503810528), (62, 0.1334311702675352), (434, 0.12923031627427153), (121, 0.12895469745158045)]
[(160, 0.19156208773443828), (82, 0.17331073340920658), (121, 0.17175131608730435), (107, 0.1583726855711284), (62, 0.1483661159857358)]


In [ ]:
# UPDATE AUTHS
auths = links_formatted.join(hubs).map(lambda x: (x[1][0], x[1][1])).reduceByKey(lambda x, y: x + y)

print(auths.count())   
print(auths.take(3))    

991
[(316, 38.0), (268, 29.0), (380, 33.0)]


In [ ]:
# UPDATE HUBS
hubs = links_formatted.map(lambda x: (x[1], x[0])).join(auths).map(lambda x: (x[1][0], x[1][1])).reduceByKey(lambda x, y: x + y)

print(hubs.count())   
print(hubs.take(3))    


868
[(6, 5180.0), (438, 4400.0), (132, 4540.0)]


In [ ]:
# NORMALIZE
def normalize(rdd):
  temp = rdd.map(lambda x: ('sum', x[1]**2)).reduceByKey(lambda x, y: x+y)

  norm = temp.collect()[0][1] ** 0.5
  updated = rdd.map(lambda x: (x[0], x[1] / norm))
  updated = updated.sortBy(lambda x: x[1], ascending=False)

  return updated

auths = normalize(auths)
hubs = normalize(hubs)

print(auths.take(3))
print(hubs.take(3))

print(auths.count())
print(hubs.count())


[(160, 0.1769052195768834), (62, 0.1493680863408591), (107, 0.14102350051176082)]
[(160, 0.18994196271725522), (82, 0.16127271673704585), (121, 0.16024834606070307)]
991
868


In [ ]:
path_links = '/content/drive/My Drive/Coursework/CS 5683_Big_Data/Assignment-6/links-sorted-small.txt'
links = sc.textFile(path_links)

links_formatted = links.map(lambda x: (int(x.split(':')[0]),
        [int(y) for y in x.split(':')[1].split(' ') if y != '']))

print(links_formatted.count())
links_formatted.take(3)

2000


[(150139, [150145]), (150140, [150145]), (150141, [150145])]

In [ ]:
def f(x): return x
out_links = links_formatted.flatMapValues(f)

print(out_links.count())
out_links.take(3)

34130


[(150139, 150145), (150140, 150145), (150141, 150145)]

In [ ]:
out_links = links_formatted.flatMapValues(lambda x: [y for y in x]).filter(lambda x: x[1] != [])

print(out_links.count())
out_links.take(2)                   

34130


[(150139, 150145), (150140, 150145)]

In [ ]:
path_titles = '/content/drive/My Drive/Coursework/CS 5683_Big_Data/Assignment-6/titles-sorted-small.txt'

titles = sc.textFile(path_titles)
titles_indexed = titles.zipWithIndex().map(lambda x: (x[1] + 1, x[0]))

titles_indexed.take(5)

[(1, '!'), (2, '!!'), (3, '!!!'), (4, '!!!!'), (5, '!!!Fuck_You!!!')]

In [ ]:
links, titles = read_txt(path_links, path_titles)
print('\n\nTABLES CREATED\n\n')

# Remove dead links
out_links = remove_dead_links(links)

# Create auths and hubs
auths, hubs = create_auths_hubs(titles)

In [ ]:
print(out_links.count())
print(auths.count())

34130
400000


In [ ]:
out_links.take(3)

[(150139, 150145), (150140, 150145), (150141, 150145)]

In [ ]:
auths = out_links.join(hubs).map(lambda x: (x[1][0], x[1][1])).reduceByKey(lambda x, y: x + y)

auths.take(4)

[(149668, 3.0), (146592, 7.0), (146892, 7.0), (146968, 7.0)]

In [ ]:
hubs = out_links.map(lambda x: (x[1], x[0])).join(auths).map(lambda x: (x[1][0], x[1][1])).reduceByKey(lambda x, y: x + y)

hubs.take(3)

[(151248, 115.0), (152076, 259.0), (150462, 1489.0)]

In [ ]:
from pyspark import SparkContext
import time


def read_txt(path_links, path_titles):
    # Read
    links = sc.textFile(path_links)
    titles = sc.textFile(path_titles)

    # Split links
    links_formatted = links.map(lambda x: (int(x.split(':')[0]),
        [int(y) for y in x.split(':')[1].split(' ') if y != '']))

    # Zip titles
    titles_indexed = titles.zipWithIndex().map(lambda x: (x[1] + 1, x[0]))
    titles_indexed.cache()

    return links_formatted, titles_indexed


def remove_dead_links(links_rdd):
    links_rdd.repartition(32)
    links_rdd.cache()
    print("\n\nREMOVING DEAD INKS\n\n")
    out_links = links_rdd.flatMapValues(lambda x: [y for y in x]) \
                    .filter(lambda x: x[1] != [])
    print("\n\nREMOVING LINKS - DONE\n\n")
    out_links.cache()
    return out_links

def create_auths_hubs(titles_indexed):
    auths = titles_indexed.map(lambda x: (x[0], 1.0))
    print(auths.take(10))
    print('\nAuths RDD created\n')
    hubs = auths
    print(hubs.take(10))
    print('\nHubs RDD created\n')
    return auths, hubs

# UPDATE AUTHS
def update_auths(hubs):
    auths = out_links.join(hubs) \
            .map(lambda x: (x[1][0], x[1][1])) \
            .reduceByKey(lambda x, y: x + y)
    auths.cache()
    return auths

# UPDATE HUBS
def update_hubs(auths):
    hubs = out_links.map(lambda x: (x[1], x[0])) \
            .join(auths) \
            .map(lambda x: (x[1][0], x[1][1])) \
            .reduceByKey(lambda x, y: x + y)
    hubs.cache()
    return hubs

def normalize(rdd):
  temp = rdd.map(lambda x: ('sum', x[1]**2)).reduceByKey(lambda x, y: x+y)

  norm = temp.collect()[0][1] ** 0.5
  updated = rdd.map(lambda x: (x[0], x[1] / norm))
  updated = updated.sortBy(lambda x: x[1], ascending=False)

  updated.cache()

  return updated

# NORMALIZE
# def normalize(rdd):
#     temp = rdd.map(lambda x: ('sum', x[1]**2)) \
#             .reduceByKey(lambda x, y: x+y)
#     norm = temp.collect()[0][1] ** 0.5
#     updated = rdd.map(lambda x: (x[0], x[1] / norm))
#     updated = updated.sortBy(lambda x: x[1], ascending=False)
#     updated.cache()
#     return updated

# CONVERT OUTPUT
def print_output(list_output, titles):
    index_titles = [x[0] for x in list_output]
    titles_output = titles.filter(lambda x: x[0] in index_titles).collect()
    print('\nPAGE ID AND SCORE')
    for item in list_output:
        print(item)
    print('\nPAGE ID AND TITLE')
    for item in titles_output:
        print(item)

# UPDATE PROCESS
def iterate_update(hubs, n_iter):
    # In each iteration, update auths, update hubs and normalize both
    for i in range(n_iter):
        start = time.time()
        print('\n===== ITERATION %s - UPDATING AUTHS =====' % (i + 1))
        auths = update_auths(hubs)
        print(auths.take(3))
        print('\n===== ITERATION %s - UPDATING HUBS =====' % (i + 1))
        hubs = update_hubs(auths)
        print('\n===== ITERATION %s - NORMALIZING AUTHS =====' % (i + 1))
        auths = normalize(auths)
        print('\n===== ITERATION %s - NORMALIZING HUBS =====' % (i + 1))
        hubs = normalize(hubs)
        elapsed = time.time() - start
        print('Elapsed time: ', elapsed)
        # if i in [0,7]:
        #     print('\n\n===== ITERATION %s - AUTHS OUTPUT =====\n\n' % (i + 1))
        #     print_output(auths.takeOrdered(20, key=lambda x: -x[1]), titles)
        #     print('\n\n===== ITERATION %s - HUBS OUTPUT =====\n\n' % (i + 1))
        #     print_output(hubs.takeOrdered(20, key=lambda x: -x[1]), titles)



if __name__ == "__main__":

    # sc = SparkContext()

    # Read and format input
    path_links = '/content/drive/My Drive/Coursework/CS 5683_Big_Data/Assignment-6/link-sorted-small.txt'
    path_titles = '/content/drive/My Drive/Coursework/CS 5683_Big_Data/Assignment-6/titles-sorted-small.txt'
    #links, titles = read_txt(path_links, path_titles)
    print('\n\nTABLES CREATED\n\n')

    # Remove dead links
    #out_links = remove_dead_links(links)

    path_links = '/content/drive/My Drive/Coursework/CS 5683_Big_Data/Assignment-6/email-Eu-core.txt'
    links = sc.textFile(path_links)

    out_links = links.map(lambda x: (int(x.split(' ')[0]),int(x.split(' ')[1])))

    # Create auths and hubs
    #auths, hubs = create_auths_hubs(titles)
    auths = out_links.flatMap(lambda x: x).distinct().map(lambda x: (x, 1.0))
    hubs = auths

    # Iterate updating
    iterate_update(hubs, 8)

    




TABLES CREATED



===== ITERATION 1 - UPDATING AUTHS =====
[(316, 38.0), (268, 29.0), (380, 33.0)]

===== ITERATION 1 - UPDATING HUBS =====

===== ITERATION 1 - NORMALIZING AUTHS =====

===== ITERATION 1 - NORMALIZING HUBS =====
Elapsed time:  3.1851754188537598

===== ITERATION 2 - UPDATING AUTHS =====
[(0, 1.1975418524742887), (248, 0.7257272253167064), (560, 1.1877577992194759)]

===== ITERATION 2 - UPDATING HUBS =====

===== ITERATION 2 - NORMALIZING AUTHS =====

===== ITERATION 2 - NORMALIZING HUBS =====
Elapsed time:  5.223223447799683

===== ITERATION 3 - UPDATING AUTHS =====
[(0, 1.1169936729536085), (12, 2.1918100849682496), (264, 0.8046423394254673)]

===== ITERATION 3 - UPDATING HUBS =====

===== ITERATION 3 - NORMALIZING AUTHS =====

===== ITERATION 3 - NORMALIZING HUBS =====
Elapsed time:  7.121347904205322

===== ITERATION 4 - UPDATING AUTHS =====
[(0, 1.1050388273186436), (560, 1.1464313998435476), (368, 1.9124350296452675)]

===== ITERATION 4 - UPDATING HUBS =====

==